In [41]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Intraday Double RSI backtest — with Trend & Confirmation Filters
- LONGS only above EMA200 (toggleable)
- Confirmation: ADX(14) >= adx_min OR RSI5 slope condition (toggleable)
- Next-bar OPEN entries, SL/TP exits (percent OR rupee total-PnL), forced EOD square-off
- Groww intraday brokerage per trade
- Saves trade log CSV with GrossPnL, Charges, NetPnL

Requires: pandas, numpy, yfinance
"""

import os, math, datetime as dt
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception as e:
    raise SystemExit("Please: pip install yfinance pandas numpy") from e


# =========================
# CONFIG
# =========================
@dataclass
class Config:
    symbols: List[str] = ('ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS',
             'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS',
             'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
             'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS',
             'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS',
             'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS',
             'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS',
             'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS',
             'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TMPV.NS',
             'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS')
    period_5m: str = "60d"
    tz: str = "Asia/Kolkata"

    # Entry window (entries allowed only within this; exits/EOD always allowed)
    trade_window: Optional[Tuple[str, str]] = ("10:00","14:30")  # e.g., ("09:30","14:00")
    mkt_start: str = "09:15"
    mkt_end: str   = "15:25"  # last usable 5m candle before 15:30

    # Core RSI signal
    rsi_len: int = 14
    buy_rsi5_lt: float = 30.0
    buy_rsi60_gt: float = 50.0
    sell_rsi5_gt: float = 70.0
    sell_rsi60_lt: float = 50.0

    enable_longs: bool = True
    enable_shorts: bool = True  # EMA200 trend filter applies only to LONGS

    # --- NEW FILTERS ---
    use_trend_filter: bool = True      # only LONG above EMA200
    ema_trend_len: int = 200

    use_adx_filter: bool = True
    adx_len: int = 14
    adx_min: float = 20.0              # typical 18–22 range; default 20

    use_rsi_slope: bool = True         # RSI5 rising for longs; falling for shorts

    # Capital / risk
    cash_per_symbol: float = 100_000.0
    leverage: float = 5.0  # 5× buying power → ₹5L notional per trade

    # Choose EITHER % stops or ₹ stops (₹ = TOTAL trade PnL)
    sl_pct: Optional[float] = None        # e.g., 0.01 for 1% SL; set None to use rupees
    tp_pct: Optional[float] = None        # e.g., 0.02 for 2% TP; set None to use rupees
    sl_rupees: Optional[float] = 700.0   # TOTAL rupees per trade (converted to per-share using qty)
    tp_rupees: Optional[float] = 2000.0   # TOTAL rupees per trade

    # Friction
    slippage_pct: float = 0.0000          # 2 bps per fill; brokerage modeled separately

    # Output
    out_dir: str = "outputs/intraday_double_rsi"
    out_prefix: str = "trades"

CFG = Config()


# =========================
# Groww intraday brokerage (as provided)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5
    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)
    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover
    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover
    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover
    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)
    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover
    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)
    return {"total_charges": total}


# =========================
# Helpers: data, sessions, indicators
# =========================
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def download_5m(symbols: List[str], period: str, tz: str) -> Dict[str, pd.DataFrame]:
    out = {}
    for s in symbols:
        df = yf.download(s, period=period, interval="5m",
                         auto_adjust=False, threads=False, progress=False, prepost=False, multi_level_index=False)
        if df.empty:
            print(f"[WARN] No data for {s}")
            continue
        df = df.dropna(how="any")
        df.index = df.index.tz_localize(tz) if df.index.tz is None else df.index.tz_convert(tz)
        out[s] = df
    return out

def session_mask(idx: pd.DatetimeIndex, hhmm_start: str, hhmm_end: str, tz: str) -> pd.Series:
    t = idx.tz_convert(tz).time
    sh, sm = map(int, hhmm_start.split(":")); eh, em = map(int, hhmm_end.split(":"))
    start = pd.to_datetime(f"{sh:02d}:{sm:02d}").time()
    end   = pd.to_datetime(f"{eh:02d}:{em:02d}").time()
    s = pd.Series(t, index=idx)
    return (s >= start) & (s <= end)

def last_bar_mask(idx: pd.DatetimeIndex, tz: str) -> pd.Series:
    d = idx.tz_convert(tz).date
    s = pd.Series(d, index=idx)
    return ~s.duplicated(keep="last")

def rsi(series: pd.Series, length: int) -> pd.Series:
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50)

def ema(series: pd.Series, length: int) -> pd.Series:
    return series.ewm(span=length, adjust=False).mean()

def adx(df: pd.DataFrame, length: int = 14) -> pd.Series:
    """Wilder's ADX with full DateTimeIndex alignment (no length mismatch)."""
    high, low, close = df["High"], df["Low"], df["Close"]

    # True Range
    tr = pd.concat([
        (high - low),
        (high - close.shift()).abs(),
        (low  - close.shift()).abs()
    ], axis=1).max(axis=1)

    # Directional Movement (as Series with df.index)
    up_move   = high.diff()
    down_move = -low.diff()
    plus_dm  = pd.Series(np.where((up_move >  down_move) & (up_move > 0),  up_move,  0.0), index=df.index)
    minus_dm = pd.Series(np.where((down_move > up_move)  & (down_move > 0), down_move, 0.0), index=df.index)

    # Wilder smoothing via EMA(alpha=1/n), keep index
    atr      = tr.ewm(alpha=1/length, adjust=False, min_periods=length).mean()
    plus_di  = 100 * plus_dm.ewm(alpha=1/length, adjust=False, min_periods=length).mean() / atr
    minus_di = 100 * minus_dm.ewm(alpha=1/length, adjust=False, min_periods=length).mean() / atr

    dx = (100 * (plus_di - minus_di).abs() / (plus_di + minus_di)).replace([np.inf, -np.inf], np.nan)
    adx_val = dx.ewm(alpha=1/length, adjust=False, min_periods=length).mean()

    # Clean up NaNs at the start
    return adx_val.fillna(method="bfill").fillna(0.0)


def compute_indicators_5m(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Core RSI
    out["RSI5"] = rsi(out["Close"], CFG.rsi_len)
    rsi60 = rsi(out["Close"].resample("60min").last(), CFG.rsi_len).reindex(out.index, method="ffill")
    out["RSI60"] = rsi60
    # RSI slope (first difference)
    out["RSI5_SLOPE"] = out["RSI5"].diff()
    # Trend filter: EMA200
    out["EMA200"] = ema(out["Close"], CFG.ema_trend_len)
    # ADX
    out["ADX"] = adx(out, CFG.adx_len)
    return out

def next_bar_open_price(df: pd.DataFrame) -> pd.Series:
    return df["Open"].shift(-1)

def price_with_slippage(price: float, side: str) -> float:
    slip = CFG.slippage_pct
    if side == "buy":
        return price * (1 + slip)
    if side == "sell":
        return price * (1 - slip)
    return price


# =========================
# Entry logic with filters
# =========================
def decide_entry(side: str, row: pd.Series) -> bool:
    """Return True if entry conditions are met for the given side on this bar (evaluate at close)."""
    # Core RSI conditions
    if side == "long":
        core_ok = (row["RSI60"] > CFG.buy_rsi60_gt) and (row["RSI5"] < CFG.buy_rsi5_lt)
    else:
        core_ok = (row["RSI60"] < CFG.sell_rsi60_lt) and (row["RSI5"] > CFG.sell_rsi5_gt)

    if not core_ok:
        return False

    # Trend filter (applies only to LONGs)
    if side == "long" and CFG.use_trend_filter:
        if not (row["Close"] > row["EMA200"]):
            return False

    # Confirmation: ADX or RSI5 slope (at least one)
    conf_ok = True
    wants_conf = CFG.use_adx_filter or CFG.use_rsi_slope
    if wants_conf:
        adx_ok = (row["ADX"] >= CFG.adx_min) if CFG.use_adx_filter else False
        if CFG.use_rsi_slope:
            slope_ok = (row["RSI5_SLOPE"] > 0) if side == "long" else (row["RSI5_SLOPE"] < 0)
        else:
            slope_ok = False
        conf_ok = (adx_ok or slope_ok)

    return conf_ok


# =========================
# Backtest per symbol (rupee stops = total-PnL)
# =========================
def backtest_symbol(df_raw: pd.DataFrame, symbol: str) -> pd.DataFrame:
    if df_raw.empty:
        return pd.DataFrame()

    # Restrict to market session
    in_session = session_mask(df_raw.index, CFG.mkt_start, CFG.mkt_end, CFG.tz)
    df = df_raw.loc[in_session].copy()
    if df.empty:
        return pd.DataFrame()

    # Entry window
    tw = session_mask(df.index, CFG.trade_window[0], CFG.trade_window[1], CFG.tz) if CFG.trade_window else pd.Series(True, index=df.index)

    # Indicators & helpers
    df = compute_indicators_5m(df)
    df["EOD"] = last_bar_mask(df.index, CFG.tz)
    df["NextOpen"] = next_bar_open_price(df)

    # Position state
    in_pos = False
    pos_side = None                 # "long" / "short"
    qty = 0
    entry_px = np.nan
    entry_time = None

    # Capital
    notional = CFG.cash_per_symbol * CFG.leverage

    # Logs
    logs = []

    def close_position(exit_px: float, exit_time, reason: str):
        nonlocal in_pos, pos_side, qty, entry_px, entry_time
        if not in_pos or qty == 0 or pd.isna(exit_px):
            return

        # Brokerage turnovers
        if pos_side == "long":
            buy_turnover  = qty * entry_px
            sell_turnover = qty * exit_px
        else:
            buy_turnover  = qty * exit_px
            sell_turnover = qty * entry_px

        charges = groww_intraday_charges(buy_turnover, sell_turnover)["total_charges"]

        pnl_gross = (exit_px - entry_px) * qty if pos_side == "long" else (entry_px - exit_px) * qty
        pnl_net = pnl_gross - charges
        ret = pnl_gross / (entry_px * abs(qty)) if entry_px and qty else 0.0

        logs.append({
            "Symbol": symbol,
            "Side": "Long" if pos_side == "long" else "Short",
            "Entry Time": entry_time,
            "Exit Time": exit_time,
            "Qty": qty,
            "Entry": round(entry_px, 2),
            "Exit": round(exit_px, 2),
            "GrossPnL": round(pnl_gross, 2),
            "Charges": round(charges, 2),
            "NetPnL": round(pnl_net, 2),
            "Ret%": round(ret * 100.0, 2),
            "Exit Reason": reason
        })

        # Reset
        in_pos = False
        pos_side = None
        qty = 0
        entry_px = np.nan

    # Loop bars
    for ts, row in df.iterrows():
        # ========= EXIT CHECKS (if already in a position) =========
        if in_pos:
            hi = row["High"]; lo = row["Low"]

            if CFG.sl_pct is not None and CFG.tp_pct is not None:
                # Percent mode → per-share distances from entry price
                sl_dist_ps = entry_px * CFG.sl_pct
                tp_dist_ps = entry_px * CFG.tp_pct
            else:
                # Rupee mode → TOTAL per trade → convert to PER-SHARE using qty
                sl_total = CFG.sl_rupees if CFG.sl_rupees is not None else np.inf
                tp_total = CFG.tp_rupees if CFG.tp_rupees is not None else np.inf
                sl_dist_ps = sl_total / max(1, qty)
                tp_dist_ps = tp_total / max(1, qty)

            if pos_side == "long":
                sl_level = entry_px - sl_dist_ps
                tp_level = entry_px + tp_dist_ps
                hit_sl = lo <= sl_level
                hit_tp = hi >= tp_level
                if hit_sl and hit_tp:
                    exit_px = price_with_slippage(sl_level, "sell")  # SL-first tie policy
                    close_position(exit_px, ts, "SL&TP(SL-first)")
                elif hit_sl:
                    exit_px = price_with_slippage(sl_level, "sell")
                    close_position(exit_px, ts, "SL")
                elif hit_tp:
                    exit_px = price_with_slippage(tp_level, "sell")
                    close_position(exit_px, ts, "TP")
            else:
                sl_level = entry_px + sl_dist_ps
                tp_level = entry_px - tp_dist_ps
                hit_sl = hi >= sl_level
                hit_tp = lo <= tp_level
                if hit_sl and hit_tp:
                    exit_px = price_with_slippage(sl_level, "buy")
                    close_position(exit_px, ts, "SL&TP(SL-first)")
                elif hit_sl:
                    exit_px = price_with_slippage(sl_level, "buy")
                    close_position(exit_px, ts, "SL")
                elif hit_tp:
                    exit_px = price_with_slippage(tp_level, "buy")
                    close_position(exit_px, ts, "TP")

        # ========= EOD FORCED SQUARE-OFF =========
        if in_pos and row["EOD"]:
            side = "sell" if pos_side == "long" else "buy"
            exit_px = price_with_slippage(row["Close"], side)
            close_position(exit_px, ts, "EOD")

        # ========= ENTRY (next-bar OPEN fill) =========
        if (not in_pos) and tw.loc[ts] and pd.notna(row["NextOpen"]):
            # LONG side
            if CFG.enable_longs and decide_entry("long", row):
                fill = price_with_slippage(row["NextOpen"], "buy")
                qty = int(math.floor((notional) / fill))
                if qty > 0:
                    entry_px = fill
                    in_pos = True
                    pos_side = "long"
                    entry_time = ts
            # SHORT side (no EMA filter by default; uses ADX/RSI-slope if enabled)
            elif CFG.enable_shorts and decide_entry("short", row):
                fill = price_with_slippage(row["NextOpen"], "sell")
                qty = int(math.floor((notional) / fill))
                if qty > 0:
                    entry_px = fill
                    in_pos = True
                    pos_side = "short"
                    entry_time = ts

    # Final safety close
    if in_pos:
        last_ts = df.index[-1]
        side = "sell" if pos_side == "long" else "buy"
        exit_px = price_with_slippage(df.iloc[-1]["Close"], side)
        close_position(exit_px, last_ts, "EOD")

    return pd.DataFrame(logs)


# =========================
# Run & Save CSV
# =========================
def run():
    os.makedirs(CFG.out_dir, exist_ok=True)
    stamp = dt.datetime.now().strftime("%Y-%m-%d_%H%M%S")
    csv_path = os.path.join(CFG.out_dir, f"{CFG.out_prefix}_{stamp}.csv")

    data = download_5m(CFG.symbols, CFG.period_5m, CFG.tz)
    all_trades = []
    for s, df in data.items():
        tl = backtest_symbol(df, s)
        if not tl.empty:
            all_trades.append(tl)

    if not all_trades:
        print("No trades generated.")
        return

    trades = pd.concat(all_trades, ignore_index=True)

    # Save CSV
    trades.sort_values(["Symbol", "Entry Time"]).to_csv(csv_path, index=False)

    # Per-symbol summary (Net PnL)
    per_symbol = trades.groupby("Symbol").agg(
        Trades=("Symbol", "count"),
        GrossPnL=("GrossPnL", "sum"),
        Charges=("Charges", "sum"),
        NetPnL=("NetPnL", "sum"),
        WinRate=("NetPnL", lambda x: (x > 0).mean() * 100),
        AvgNetPnL=("NetPnL", "mean"),
        AvgRetPct=("Ret%", "mean"),
        MaxDD=("NetPnL", lambda x: (x.cumsum().cummax() - x.cumsum()).max() if len(x) else 0.0)
    ).sort_values("NetPnL", ascending=False).round(2)

    # Portfolio summary
    port = {
        "Total Trades": int(len(trades)),
        "Gross PnL (₹)": round(trades["GrossPnL"].sum(), 2),
        "Total Charges (₹)": round(trades["Charges"].sum(), 2),
        "Net PnL (₹)": round(trades["NetPnL"].sum(), 2),
        "Avg Net PnL / Trade (₹)": round(trades["NetPnL"].mean(), 2),
        "Win Rate (%)": round((trades["NetPnL"] > 0).mean() * 100, 2),
        "Best Trade Net (₹)": round(trades["NetPnL"].max(), 2),
        "Worst Trade Net (₹)": round(trades["NetPnL"].min(), 2),
    }

    # ---- PRINT ----
    for s in CFG.symbols:
        t = trades[trades["Symbol"] == s]
        if t.empty: continue
        print("\n" + "="*100)
        print(f"Trades for {s} | Cash=₹{CFG.cash_per_symbol:,.0f} | Margin={CFG.leverage}x | Notional≈₹{(CFG.cash_per_symbol*CFG.leverage):,.0f}")
        print("="*100)
        cols = ["Symbol","Side","Entry Time","Exit Time","Qty","Entry","Exit",
                "GrossPnL","Charges","NetPnL","Ret%","Exit Reason"]
        print(t[cols].to_string(index=False))

    print("\n" + "#"*100)
    print("PER-SYMBOL SUMMARY (Net PnL)")
    print("#"*100)
    print(per_symbol.to_string())

    print("\n" + "#"*100)
    print("PORTFOLIO SUMMARY")
    print("#"*100)
    for k, v in port.items():
        print(f"- {k}: {v}")

    print(f"\nCSV saved to: {csv_path}")

if __name__ == "__main__":
    run()



Trades for ADANIENT.NS | Cash=₹100,000 | Margin=5.0x | Notional≈₹500,000
     Symbol  Side                Entry Time                 Exit Time  Qty  Entry    Exit  GrossPnL  Charges  NetPnL  Ret% Exit Reason
ADANIENT.NS Short 2025-09-05 13:25:00+05:30 2025-09-05 13:35:00+05:30  219 2277.0 2280.20    -700.0   224.18 -924.18 -0.14          SL
ADANIENT.NS Short 2025-09-05 13:35:00+05:30 2025-09-05 13:45:00+05:30  219 2283.0 2286.20    -700.0   224.65 -924.65 -0.14          SL
ADANIENT.NS Short 2025-09-05 13:45:00+05:30 2025-09-05 13:55:00+05:30  218 2284.8 2288.01    -700.0   223.97 -923.97 -0.14          SL
ADANIENT.NS Short 2025-09-05 13:55:00+05:30 2025-09-05 14:00:00+05:30  218 2288.0 2291.21    -700.0   224.22 -924.22 -0.14          SL
ADANIENT.NS Short 2025-09-05 14:00:00+05:30 2025-09-05 14:35:00+05:30  218 2290.9 2281.73    2000.0   224.26 1775.74  0.40          TP
ADANIENT.NS Short 2025-09-17 10:55:00+05:30 2025-09-17 12:45:00+05:30  207 2409.8 2400.14    2000.0   224.06 1775.94